# Friday (prototype 2)
A dynamic NN based assistant and framework

rewritten with a Dynamic NN framework to make my life a little easier. If you would like to see a half built version in tensorflow checkout friday.ipynb

NOTE: quite a bit of this notebook rips off https://github.com/ml-jku/hopfield-layers/blob/master/examples/bit_pattern/bit_pattern_demo.ipynb

This relies on the non pip package hopefield-layers

In [1]:
!pip3 install git+https://github.com/ml-jku/hopfield-layers

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ml-jku/hopfield-layers to /tmp/pip-req-build-fme36buv
  Running command git clone --filter=blob:none --quiet https://github.com/ml-jku/hopfield-layers /tmp/pip-req-build-fme36buv
  Resolved https://github.com/ml-jku/hopfield-layers to commit f56f929c95b77a070ae675ea4f56b6d54d36e730
  Preparing metadata (setup.py) ... done


In [2]:
# Import general modules used e.g. for plotting.
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import sys
import torch
import numpy as np


# Importing Hopfield-specific modules.
from hflayers import Hopfield

# Import auxiliary modules.
from distutils.version import LooseVersion
from typing import List, Tuple

# Importing PyTorch specific modules.
from torch import Tensor
from torch.nn import Flatten, Linear, Module
from torch.nn.functional import binary_cross_entropy_with_logits
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.model_selection import train_test_split

sns.set()
sns.set_theme(style="dark") # prioritys lol

## Create Auxiliaries
Before digging into Hopfield-based networks, a few auxiliary variables and functions need to be defined. This is nothing special with respect to Hopfield-based networks, but rather common preparation work of (almost) every machine learning setting (e.g. definition of a data loader as well as a training loop). We will see, that this comprises the most work of this whole demo.

In [3]:
input_shape = [3, 3] # the input shape used in this notebook
output_shape = [3, 3] # the output shape used in this notebook
topic_shape = [3, 3]
zeros = torch.zeros(input_shape)
ones = torch.ones(input_shape)

In [4]:
#device = torch.device(r'cuda:0' if torch.cuda.is_available() else r'cpu')
device = torch.device('cpu')

In [5]:
#bit_pattern_set = BitPatternSet(
#    num_bags=2048,
#    num_instances=16,
#    num_signals=8,
#    num_signals_per_bag=1,
#    num_bits=8)
log_dir = f'resources/'
os.makedirs(log_dir, exist_ok=True)

In [6]:
def train_epoch(network: Module,
                optimiser: AdamW,
                data_loader: DataLoader
               ) -> Tuple[float, float]:
    """
    Execute one training epoch.
    
    :param network: network instance to train
    :param optimiser: optimiser instance responsible for updating network parameters
    :param data_loader: data loader instance providing training data
    :return: tuple comprising training loss as well as accuracy
    """
    network.train()
    losses, accuracies = [], []
    for sample_data in data_loader:
        data, target = sample_data[r'data'], sample_data[r'target']
        data, target = data.to(device=device), target.to(device=device)

        # Process data by Hopfield-based network.
        model_output = network.forward(input=data)

        # Update network parameters.
        optimiser.zero_grad()
        loss = binary_cross_entropy_with_logits(input=model_output, target=target, reduction=r'mean')
        loss.backward()
        clip_grad_norm_(parameters=network.parameters(), max_norm=1.0, norm_type=2)
        optimiser.step()

        # Compute performance measures of current model.
        accuracy = (model_output.sigmoid().round() == target).to(dtype=torch.float32).mean()
        accuracies.append(accuracy.detach().item())
        losses.append(loss.detach().item())
    
    # Report progress of training procedure.
    return (sum(losses) / len(losses), sum(accuracies) / len(accuracies))


def eval_iter(network: Module,
              data_loader: DataLoader
             ) -> Tuple[float, float]:
    """
    Evaluate the current model.
    
    :param network: network instance to evaluate
    :param data_loader: data loader instance providing validation data
    :return: tuple comprising validation loss as well as accuracy
    """
    network.eval()
    with torch.no_grad():
        losses, accuracies = [], []
        for sample_data in data_loader:
            data, target = sample_data[r'data'], sample_data[r'target']
            data, target = data.to(device=device), target.to(device=device)

            # Process data by Hopfield-based network.
            model_output = network.forward(input=data)
            loss = binary_cross_entropy_with_logits(input=model_output, target=target, reduction=r'mean')

            # Compute performance measures of current model.
            accuracy = (model_output.sigmoid().round() == target).to(dtype=torch.float32).mean()
            accuracies.append(accuracy.detach().item())
            losses.append(loss.detach().item())

        # Report progress of validation procedure.
        return (sum(losses) / len(losses), sum(accuracies) / len(accuracies))


def operate(network: Module,
            optimiser: AdamW,
            data_loader_train: DataLoader,
            data_loader_eval: DataLoader,
            num_epochs: int = 1
           ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Train the specified network by gradient descent using backpropagation.
    
    :param network: network instance to train
    :param optimiser: optimiser instance responsible for updating network parameters
    :param data_loader_train: data loader instance providing training data
    :param data_loader_eval: data loader instance providing validation data
    :param num_epochs: amount of epochs to train
    :return: data frame comprising training as well as evaluation performance
    """
    losses, accuracies = {r'train': [], r'eval': []}, {r'train': [], r'eval': []}
    for epoch in range(num_epochs):
        
        # Train network.
        performance = train_epoch(network, optimiser, data_loader_train)
        losses[r'train'].append(performance[0])
        accuracies[r'train'].append(performance[1])
        
        # Evaluate current model.
        performance = eval_iter(network, data_loader_eval)
        if epoch % 5 == 0:
            print("---------------------------------")
            print("epoch:", epoch, "of", num_epochs, "\naccuracy:", performance[1], "\nloss:", performance[0])
        losses[r'eval'].append(performance[0])
        accuracies[r'eval'].append(performance[1])
    
    # Report progress of training and validation procedures.
    return pd.DataFrame(losses), pd.DataFrame(accuracies)

In [7]:
def plot_performance(loss: pd.DataFrame,
                     accuracy: pd.DataFrame,
                     log_file: str
                    ) -> None:
    """
    Plot and save loss and accuracy.
    
    :param loss: loss to be plotted
    :param accuracy: accuracy to be plotted
    :param log_file: target file for storing the resulting plot
    :return: None
    """
    fig, ax = plt.subplots(1, 2, figsize=(20, 7))
    
    loss_plot = sns.lineplot(data=loss, ax=ax[0])
    loss_plot.set(xlabel=r'Epoch', ylabel=r'Cross-entropy Loss')
    
    accuracy_plot = sns.lineplot(data=accuracy, ax=ax[1])
    accuracy_plot.set(xlabel=r'Epoch', ylabel=r'Accuracy')
    
    ax[1].yaxis.set_label_position(r'right')
    fig.tight_layout()
    fig.savefig(log_file)
    plt.show(fig)

## The Friday DataFrame class


In [39]:
def string_tensor(w):
    return torch.ByteTensor(list(bytes(w, 'utf8')))

In [29]:
class FridayDataset(Dataset):
    def __init__(self, df, labels):
        super().__init__()
        self.df = df
        self.topic = labels[0]
        self.fact = labels[1]

    def __len__(self):
        # print len(self.landmarks_frame)
        # return len(self.landmarks_frame)
        return len(df.index)

    def __getitem__(self, idx):
        return {r'data': string_tensor(self.df.at[idx, self.topic]),  r'target': string_tensor(self.df.at[idx, self.fact])}


## Build the Friday model

In [30]:
class Friday(Module):
    def __init__(self, input_size, num_instances):
        super().__init__()
        self.hopfield = Hopfield(
            input_size=input_size,
            hidden_size=8,
            num_heads=8,
            update_steps_max=3,
            scaling=0.25)
        self.flatten = Flatten()
        self.output_projection = Linear(in_features=self.hopfield.output_size * num_instances, out_features=1)
        self.flatten2 = Flatten(start_dim=0)
    def forward(self, input):
        x = input
        x = self.hopfield(x)
        x = self.flatten(x)
        x = self.output_projection(x)
        x = self.flatten2(x)
        return x


## Run Friday

In [40]:
df = pd.read_csv("data/beavers.csv") #TODO add a 80/20 datset splitter 
labels = ["animal", "fact"]
dataset = FridayDataset(df = df, labels = labels)
data_loader = DataLoader(dataset=dataset)
string_tensor("hi")
for s in data_loader:
    print(s)


{'data': tensor([[ 98, 101,  97, 118, 101, 114]], dtype=torch.uint8), 'target': tensor([[ 99, 117, 116, 101]], dtype=torch.uint8)}
{'data': tensor([[ 98, 101,  97, 118, 101, 114]], dtype=torch.uint8), 'target': tensor([[102,  97, 115, 116]], dtype=torch.uint8)}
{'data': tensor([[ 98, 101,  97, 118, 101, 114]], dtype=torch.uint8), 'target': tensor([[ 98, 117, 105, 108, 100, 115,  32, 100,  97, 109, 115]],
       dtype=torch.uint8)}
{'data': tensor([[ 98, 101,  97, 118, 101, 114]], dtype=torch.uint8), 'target': tensor([[114, 117, 110, 110, 105, 110, 103,  32, 105, 110,  32, 116, 104, 101,
          32,  57,  48, 115]], dtype=torch.uint8)}
{'data': tensor([[ 98, 101,  97, 118, 101, 114]], dtype=torch.uint8), 'target': tensor([[105, 115,  32,  99, 111, 111, 108]], dtype=torch.uint8)}
{'data': tensor([[ 98, 101,  97, 118, 101, 114]], dtype=torch.uint8), 'target': tensor([[111, 114,  97, 110, 103, 101,  32, 116, 101, 101, 116, 104]],
       dtype=torch.uint8)}
{'data': tensor([[ 98, 101,  97

In [21]:
# Create data loader of training set.
data_loader_train = DataLoader(dataset=train_dataset, batch_size=32)

# Create data loader of validation set.
data_loader_eval = DataLoader(dataset=test_dataset, batch_size=32)

NameError: name 'train_dataset' is not defined

In [41]:
network = Friday(input_size = 8, num_instances = len(dataset)).to(device=device)
optimiser = AdamW(params=network.parameters(), lr=1e-3)
print(network)

Friday(
  (hopfield): Hopfield(
    (association_core): HopfieldCore(
      (out_proj): Linear(in_features=64, out_features=8, bias=True)
    )
    (norm_stored_pattern): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (norm_state_pattern): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (norm_pattern_projection): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (output_projection): Linear(in_features=160, out_features=1, bias=True)
  (flatten2): Flatten(start_dim=0, end_dim=-1)
)


In [42]:
losses, accuracies = operate(
    network=network,
    optimiser=optimiser,
    data_loader_train=data_loader,
    data_loader_eval=data_loader,
    num_epochs=500)

IndexError: tuple index out of range

In [ ]:
plot_performance(loss=losses, accuracy=accuracies, log_file=f'{log_dir}/hopfield_base.pdf')